In [61]:
# import sys
# !{sys.executable} -m pip install pandas;

In [99]:
import pandas as pd
import time

In [84]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv"))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df["latency"] = above_threshold_df["finishTimestamp"] - above_threshold_df["blockTimestamp"]

In [101]:
# Limit to hour span
hour_limit = 24
sec_limit = 24 * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

## Best opportunities so far

In [102]:
df.sort_values(by="profit", ascending=0).head(10)

,blockNumber,blockTimestamp,finishTimestamp,name,profit,optimalAmount,humanTimestamp,latency
2936,11512536,1608762382,1608762385,wbtcweth_ssuni_o,1.22700,321.700,2020-12-23 22:26:22,NaN
2116,11511099,1608743524,1608743548,snxweth_ssuni_o,0.19320,27.700,2020-12-23 17:12:04,24.0
3027,11512564,1608762753,1608762761,linkweth_ssuni_o,0.15600,47.220,2020-12-23 22:32:33,NaN
3003,11512556,1608762674,1608762677,sushiweth_ssuni_r,0.14660,8.660,2020-12-23 22:31:14,NaN
3007,11512557,1608762676,1608762707,sushiweth_ssuni_r,0.13180,8.217,2020-12-23 22:31:16,NaN
1402,11510464,1608735152,1608735183,snxweth_ssuni_o,0.11900,21.640,2020-12-23 14:52:32,31.0
1323,11510284,1608732664,1608732669,snxweth_ssuni_r,0.06188,15.340,2020-12-23 14:11:04,5.0
3006,11512557,1608762676,1608762707,linkweth_ssuni_o,0.05813,28.860,2020-12-23 22:31:16,NaN
3022,11512562,1608762735,1608762740,snxweth_ssuni_o,0.05521,14.370,2020-12-23 22:32:15,NaN
2978,11512549,1608762550,1608762558,snxweth_ssuni_o,0.05521,14.370,2020-12-23 22:29:10,NaN


## Statistics

In [103]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
threshold_profit = 0.015
above_threshold_df = df[df.profit > threshold_profit]
opp_count = above_threshold_df.blockNumber.count()
mean_latency = above_threshold_df.latency.mean()
max_profit = above_threshold_df.profit.max()
mean_profit = above_threshold_df.profit.mean()
total_profit = above_threshold_df.profit.sum()

In [104]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {mean_latency:.0f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")
print(f"Summed profit: {total_profit:.2f} ETH")

Opportunitys count: 146
Latency average: 15 sec
Max profit: 1.23 ETH
Average profit: 0.05 ETH
Summed profit: 7.03 ETH


## Instruction analysis

In [106]:
opp_comp_df = df.groupby("name").agg({"blockNumber": "count", "profit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
wbtcweth_ssuni_o,9,1.227000,1.262725,0.140303
amplweth_ssuni_r,46,0.035450,1.456979,0.031673
snxweth_ssuni_o,160,0.193200,2.400867,0.015005
linkweth_ssuni_o,27,0.156000,0.382889,0.014181
snxweth_ssuni_r,43,0.061880,0.401821,0.009345
sushiweth_ssuni_r,316,0.146600,2.524437,0.007989
yfiweth_ssuni_r,3,0.006979,0.015763,0.005254
sushiweth_ssuni_o,452,0.014900,1.854228,0.004102
kp3rweth_ssuni_o,115,0.023120,0.443317,0.003855
yfiweth_ssuni_o,15,0.018330,0.057010,0.003801
